# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [4]:
import pandas as pd
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

from razdel import tokenize

In [5]:
data = pd.read_csv('labeled.csv')
data.shape

(14412, 2)

In [6]:
data.head(15)

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0
5,Для каких стан является эталоном современная с...,1.0
6,В шапке были ссылки на инфу по текущему фильму...,0.0
7,УПАД Т! ТАМ НЕЛЬЗЯ СТРОИТЬ! ТЕХНОЛОГИЙ НЕТ! РА...,1.0
8,"Ебать тебя разносит, шизик.\n",1.0
9,"Обосрался, сиди обтекай\n",1.0


In [7]:
train, test = train_test_split(data, test_size=0.2, shuffle=True)

In [8]:
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [9]:
Y = train.toxic.values
Y_test = test.toxic.values

In [10]:
# векторизация с дефолтной токенизацией
def_vectorizer = CountVectorizer()
X1 = def_vectorizer.fit_transform(train.comment)
X1_test = def_vectorizer.transform(test.comment) 

In [11]:
X1.shape, X1_test.shape

((11529, 59528), (2883, 59528))

In [12]:
# обучение линейного классификатора 
clf_1 = LogisticRegression(C=0.1, class_weight='balanced',  max_iter=1000)
clf_1.fit(X1, Y)
report1 = classification_report(Y_test, clf_1.predict(X1_test), target_names=['Neutral', 'Toxic'], digits=4) 
print(report1)

              precision    recall  f1-score   support

     Neutral     0.9062    0.8332    0.8682      1901
       Toxic     0.7207    0.8330    0.7728       982

    accuracy                         0.8332      2883
   macro avg     0.8134    0.8331    0.8205      2883
weighted avg     0.8430    0.8332    0.8357      2883



In [13]:
# векторизация с токенизацией Razdel
def razdeltokenizer(text):
    tokens = list(tokenize(text))
    tokens = [_.text for _ in tokens]
    return(tokens)

cust_vectorizer = CountVectorizer(tokenizer=razdeltokenizer)
X2 = cust_vectorizer.fit_transform(train.comment)
X2_test = cust_vectorizer.transform(test.comment) 

In [14]:
X2.shape, X2_test.shape

((11529, 60347), (2883, 60347))

In [15]:
# # обучение линейного классификатора 
clf_2 = LogisticRegression(C=0.1, class_weight='balanced', max_iter=1000)
clf_2.fit(X2, Y)
report2 = classification_report(Y_test, clf_2.predict(X2_test), target_names=['Neutral', 'Toxic'], digits=4)
print(report2)

              precision    recall  f1-score   support

     Neutral     0.9030    0.8375    0.8690      1901
       Toxic     0.7241    0.8259    0.7716       982

    accuracy                         0.8335      2883
   macro avg     0.8136    0.8317    0.8203      2883
weighted avg     0.8421    0.8335    0.8358      2883



In [16]:
print("\033[1m" + "Результаты LogReg + default tokenizer:\n" + "\033[0m", report1)

print("\033[1m" + "Результаты LogReg + razdel tokenizer:\n" + "\033[0m", report2)

Результаты LogReg + default tokenizer:
               precision    recall  f1-score   support

     Neutral     0.9062    0.8332    0.8682      1901
       Toxic     0.7207    0.8330    0.7728       982

    accuracy                         0.8332      2883
   macro avg     0.8134    0.8331    0.8205      2883
weighted avg     0.8430    0.8332    0.8357      2883

Результаты LogReg + razdel tokenizer:
               precision    recall  f1-score   support

     Neutral     0.9030    0.8375    0.8690      1901
       Toxic     0.7241    0.8259    0.7716       982

    accuracy                         0.8335      2883
   macro avg     0.8136    0.8317    0.8203      2883
weighted avg     0.8421    0.8335    0.8358      2883



*__Выводы__*: Увеличив итерацию, стала заметна разница. Классификатор с токенизатором Razdel показал результаты лучше (F-мера **+0,003**, Accuracy **+0,02**, Точность **+0,01**, взвешенная полнота  **+0,01**). Пусть и не значительно, зато стабильно по всем показателям.


## Задание 2 (3 балла)

Преобразуйте таблицу с абсолютными частотностями в семинарской тетрадке в таблицу с tfidf значениями. (Таблица - https://i.ibb.co/r5Nc2HC/abs-bow.jpg) Формула tfidf есть в семинаре на картнике с пояснениями на английском. 
Считать нужно в питоне. Формат итоговой таблицы может быть любым, главное, чтобы был код и можно было воспроизвести вычисления. 

In [18]:
lines = ['я и ты', 'ты и я', 'я я и только я', 'только не я', 'он']
raws = ['я', 'ты', 'и', 'только', 'не', 'он']

# токенизируем каждое предложение
data = [razdeltokenizer(d) for d in lines]


def calculate_tf_idf(docs_tokenized, vocab):
    N = len(docs_tokenized)
    V = len(vocab)
    result = np.zeros((N, V))    
    for i in range(0, N):
        doc = docs_tokenized[i]
        for w in doc:
            j = vocab.index(w)
            tf = doc.count(w)/len(doc) # посчитали TF
            df = 0
          
            for d in docs_tokenized:
                if w in d:
                    df += 1
            idf = np.log10(N/df)        #посчитали IDF (логарифм)
            tfidf = tf * idf
            result[i, j] = round(tfidf, 3)
            
    return result
                
result = pd.DataFrame(calculate_tf_idf(data, raws), index =  lines, columns = raws)
print(result)

                    я     ты      и  только     не     он
я и ты          0.032  0.133  0.074   0.000  0.000  0.000
ты и я          0.032  0.133  0.074   0.000  0.000  0.000
я я и только я  0.058  0.000  0.044   0.080  0.000  0.000
только не я     0.032  0.000  0.000   0.133  0.233  0.000
он              0.000  0.000  0.000   0.000  0.000  0.699


## Задание 3 (2 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к классификаторам:   
а) один должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров  
в) у классификатора должно быть задано вручную как минимум 2 параметра  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

### Модель №1. Логистическая регрессия + TF-idf Vetorizer

In [ ]:
import nltk
from nltk.corpus import stopwords

In [37]:
nltk.download("stopwords") 
russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eyer8\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [53]:
vectorizer = TfidfVectorizer(lowercase=True, tokenizer=razdeltokenizer, stop_words=russian_stopwords, min_df=4, max_df=0.2)

In [54]:
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)

In [55]:
X.shape, X_test.shape

((11529, 8743), (2883, 8743))

In [56]:
lg_classifier = LogisticRegression(C=0.1, class_weight='balanced', max_iter=1000)
lg_classifier.fit(X, Y)


report = classification_report(Y_test, lg_classifier.predict(X_test), target_names=['Neutral', 'Toxic'])
print(report)

              precision    recall  f1-score   support

     Neutral       0.89      0.84      0.86      1901
       Toxic       0.72      0.79      0.75       982

    accuracy                           0.82      2883
   macro avg       0.80      0.81      0.81      2883
weighted avg       0.83      0.82      0.82      2883



In [ ]:
lg_classifier.predict_proba(X_test)


### Модель №2. Наивный байесовский классификатор + Count Vectorizer

In [62]:
vectorizer = CountVectorizer(lowercase=True, tokenizer=razdeltokenizer, stop_words=russian_stopwords, min_df=3, max_df=0.2)

In [63]:
X_v2 = vectorizer.fit_transform(train.comment)
X_test_v2 = vectorizer.transform(test.comment)

In [64]:
X_v2.shape, X_test_v2.shape

((11529, 12448), (2883, 12448))

In [67]:
NB_classifier = MultinomialNB()
NB_classifier.fit(X_v2, Y)

report = classification_report(Y_test, NB_classifier.predict(X_test_v2), target_names=['Neutral', 'Toxic'])
print(report)

              precision    recall  f1-score   support

     Neutral       0.87      0.93      0.90      1901
       Toxic       0.84      0.73      0.78       982

    accuracy                           0.86      2883
   macro avg       0.85      0.83      0.84      2883
weighted avg       0.86      0.86      0.86      2883



In [70]:
print(NB_classifier.predict_proba(X_test_v2)[:, 1])[-10:]

[2.72941821e-01 9.99922587e-01 4.80024981e-07 ... 7.61772164e-02
 9.87880026e-01 7.78256343e-01]


TypeError: 'NoneType' object is not subscriptable

### *Задание 4 (2 балла)

Для классификаторов LogisticRegression, Decision Trees, Naive Bayes, Random Forest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию.

In [ ]:
# ваш код